## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.truncated_normal([784, 128], stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([128]))
        # 第二层权重和偏置
        self.W2 = tf.Variable(tf.random.truncated_normal([128, 10], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([10]))
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 784])
        # ReLU
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        # 输出logits
        logits = tf.matmul(h1, self.W2) + self.b2
        
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [15]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 0.7281257 ; accuracy 0.8229
epoch 1 : loss 0.7272714 ; accuracy 0.82308334
epoch 2 : loss 0.72642016 ; accuracy 0.8232667
epoch 3 : loss 0.7255724 ; accuracy 0.82338333
epoch 4 : loss 0.7247279 ; accuracy 0.8235
epoch 5 : loss 0.7238867 ; accuracy 0.8236833
epoch 6 : loss 0.72304875 ; accuracy 0.82383335
epoch 7 : loss 0.722214 ; accuracy 0.8239833
epoch 8 : loss 0.7213825 ; accuracy 0.8241
epoch 9 : loss 0.7205542 ; accuracy 0.8243167
epoch 10 : loss 0.7197289 ; accuracy 0.82446665
epoch 11 : loss 0.7189069 ; accuracy 0.8246
epoch 12 : loss 0.7180879 ; accuracy 0.82483333
epoch 13 : loss 0.717272 ; accuracy 0.82505
epoch 14 : loss 0.7164593 ; accuracy 0.82525
epoch 15 : loss 0.7156495 ; accuracy 0.8254
epoch 16 : loss 0.7148429 ; accuracy 0.82555
epoch 17 : loss 0.7140392 ; accuracy 0.82563335
epoch 18 : loss 0.71323866 ; accuracy 0.8258
epoch 19 : loss 0.71244115 ; accuracy 0.82593334
epoch 20 : loss 0.71164644 ; accuracy 0.8260667
epoch 21 : loss 0.7108547 ; accuracy 